## Hausaufgabe 2 

### Abgage via studip bis spätestens 08.05. 12:00 Uhr (zur VL)! 
### Verteidigung 11.05. 08:00-10:00 Uhr (Timeslots werden vorher bekannt gegeben)!


- In jedem Blatt kann maximal 5% Bonus erreicht werden, falls:
    - Mind. 60% des Arbeitsblatts korrekt bearbeitet wurde und
    - das Arbeitsblatt erfolgreich verteidigt (den Tutoren erklärt) wurde.
    - Bonus % = 5% * Teil der Hausaufgabe erfolgreich verteidigt
    - Erfolgreiche Verteidigung: Ein von uns ausgewähltes Team-Mitglied erklärt die Lösung und beantwortet Fragen dazu.
    - ChatGPT und andere Hilfsmittel dürfen verwendet werden, müssen aber zitiert werden.
    - Code muss ohne Hilfsmittel erläutert werden
- Wir bewerten nicht nur die Korrektheit der Ergebnisse, sondern auch ob der Code effizient, einfach zu verstehen, aufgeräumt und gut dokumentiert ist.
- Geben Sie eine Lösung pro Gruppe ab und packen Sie Ihre Lösung in eine Zip-Datei mit dem Namen und Matrikelnummer eines Mitglieds. Also z.B. `mustermann_12345678.zip` 


Installatieren Sie (falls nicht vorhanden) den bash-kernel durch: `pip install bash_kernel ; python -m bash_kernel.install`

_________


In dieser Aufgabe ist das Ziel ein durch einzelne kurze Skripts einen Text zu bearbeiten sodass dieser anschließend einfacher zu verwenden ist (z.B. für Data-Science Anwendungen)

Der Text den wir im Folgenden bearbeiten werden ist das Deutsche Grundgesetz. 

## 1 a)  [2,5%]

Erstellen Sie einen neuen Ordner und laden Sie den Text herunter den Sie unter folgender Adresse finden: https://raw.githubusercontent.com/levinalex/deutsche_verfassungen/master/grundgesetz/grundgesetz.txt 

In [3]:
mkdir gesetze

In [5]:
cd gesetze

bash: cd: gesetze: No such file or directory


: 1

In [6]:
wget https://raw.githubusercontent.com/levinalex/deutsche_verfassungen/master/grundgesetz/grundgesetz.txt

--2023-05-05 15:14:02--  https://raw.githubusercontent.com/levinalex/deutsche_verfassungen/master/grundgesetz/grundgesetz.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146482 (143K) [text/plain]
Saving to: ‘grundgesetz.txt’

grundgesetz.txt     100%[===================>] 143.05K  --.-KB/s    in 0.01s   

2023-05-05 15:14:02 (11.3 MB/s) - ‘grundgesetz.txt’ saved [146482/146482]



## 1 b) [10%]
Der Text hat eine klare Struktur. Aber am Anfang findet sich ein Stück Text welches nicht in diese Struktur passt. Schreiben Sie ein einzeiliges Skript welches alles vor dem Start der Auflistung der Artikel (und Kapitel) aus dem Text löscht. Speichern Sie die Ausgabe in einer neuen Datei. 
Stellen Sie sicher dass Ihr Skript auch funktioniert wenn sich der Text am Start ändert!

In [8]:
sed -n '/^I$/,$p' grundgesetz.txt > output.txt

## 1 c) [10%]
Splitten Sie den Text in einzelne Dateien auf basierend auf den Kapiteln (z.B. "I Die Grundrechte") nummerieren Sie die Dateien.

In [9]:
declare -i countNewLines=4
fileName=""
nextIsName=false

echo "start (takes a few seconds)"

# Loops through every line
# Quote: https://stackoverflow.com/questions/1521462/looping-through-the-content-of-a-file-in-bash
while IFS="" read -r line || [ -n "$line" ]; do
    # If 4 empty lines in a row and next word isnt article -> change file name to chapter heading
    # and add the next line to the file name (I Die Grundrechte)
    # Otherwise append the line to the current file
    if [ $countNewLines -eq 4 ] && [ "${line:0:8}" != "Artikel " ]; then
        fileName=${line}
        nextIsName=true
    elif [ $nextIsName = true ]; then
        fileName="$fileName $line"
        nextIsName=false
        echo "$fileName" >> "$fileName".txt
    else
        echo $line >> "$fileName".txt
    fi
    
    # Counts empty in a row
    if [ "$line" = "" ]; then
        countNewLines+=1
    else
        countNewLines=0
    fi
done < output.txt

echo "done"

start (takes a few seconds)
done


## 1 d) [30%]
Splitten Sie jedes Kapitel in die einzelnen Artikel auf. Jeder Artikel soll in einer eigenen Datei sein welche die Nummer des Artikels als Dateinamen. Erstellen Sie für jedes Kapitel einen neuen Unterordner in dem die Artikel gesammelt werden.

In [10]:
#!/bin/bash
declare -i countNewLine=4
# Variable needed to know at what point to start writing to file after new chapter
declare -i countNewChap=0
dir=""
fileName=""

echo "start (takes a few seconds)"

# Loops through every line
# Quote: https://stackoverflow.com/questions/1521462/looping-through-the-content-of-a-file-in-bash
while IFS="" read -r line || [ -n "$line" ]; do
    # If first part of line is "Article " and it is short -> change file name to new article
    # If 4 empty lines in a row -> save line in variable "dir" and add next line to it, then create new dir
    if [ "${line:0:8}" = "Artikel " ] && [ ${#line} -lt 20 ]; then
        fileName=$line
    elif [ $countNewChap -eq 1 ]; then
        dir="$dir $line"
        mkdir "$dir"
    elif [ $countNewLine -eq 4 ]; then
        dir=$line
        countNewChap=0
    fi
    
    # If articles started and chapter heading ended
    if [ $countNewChap -gt 2 ]; then
        echo $line >> "$dir/$fileName".txt
    fi
    
    # Counts new lines in a row
    if [ "$line" = "" ]; then
        countNewLine+=1
    else
        countNewLine=0
    fi
    
    countNewChap+=1
done < output.txt

echo "done"

start (takes a few seconds)
done


## 1 e) [5%]
Löschen Sie die ursprünglichen Daten und die Daten welche Sie im Verlauf des Prozesses (abgesehen von 1d)) generiert haben. Am Ende sollen nur noch die Dateien zu den einzelnen Artikel (in getrennten Ordnern) vorhanden sein.  

In [11]:
rm *.txt

## 1 f) [2,5%]
Packen Sie den Inhalt des gesamten Ordners in komprimiertes tar-Archiv.

In [ ]:
cd ~
ls

In [ ]:
tar -czvf grundgesetz.tar.gz gesetze

## 2 a) [20%]
Schreiben Sie ein Shell-Skript, dass alle x Sekunden ausgibt ob ein durch eine PID identifizierter Prozess läuft oder nicht.
Übergeben Sie die Anzahl an Sekunden und die PID als Parameter an das Skript, sodass es wie folgt aufgerufen werden kann:

```bash
$ script [PID] [SECONDS]
```
Ihnen sind keine Einschränkungen gegeben, welche Befehle Sie dafür verwenden.

**Nützliche Befehle:** `grep`,`ps`

In [51]:
ps
chmod +x processinfo.sh

    PID TTY          TIME CMD
 101057 pts/0    00:00:00 bash
 102103 pts/0    00:00:00 ps


In [54]:

./processinfo.sh 101057 10

Running



## 2 b) [20%]
Schreiben Sie ein Shell-Skript, das die Verzeichnisstruktur eines als Parameter angegebenen Verzeichnisses auflisten kann. Schreiben Sie dazu eine rekursive Funktion, die in Unterverzeichnisse herabsteigt. 
Geben Sie das Verzeichnis und die Dateien auf dem Bildschirm aus. 
Nutzen Sie Einrückungen um die Zugehörigkeit von Dateien und Verzeichnissen zueinander zu kennzeichnen. 
Dies kann dann zum Beispiel so aussehen:

```
File: linux-3.14/COPYING
File: linux-3.14/CREDITS
Directory: linux-3.14/Documentation
    File: linux-3.14/Documentation/00-INDEX 
    Directory: linux-3.14/Documentation/ABI
        File: linux-3.14/Documentation/ABI/README 
        Directory: linux-3.14/Documentation/ABI/obsolete        
            File: linux-3.14/Documentation/ABI/obsolete/proc-sys-vm-nr_pdflush_threads 
            File: linux-3.14/Documentation/ABI/obsolete/sysfs-bus-usb
            File: linux-3.14/Documentation/ABI/obsolete/sysfs-class-rfkill
            File: linux-3.14/Documentation/ABI/obsolete/sysfs-driver-hid-roccat-koneplus 
            File: linux-3.14/Documentation/ABI/obsolete/sysfs-driver-hid-roccat-kovaplus 
            File: linux-3.14/Documentation/ABI/obsolete/sysfs-driver-hid-roccat-pyra
        Directory: linux-3.14/Documentation/ABI/removed 
            File: linux-3.14/Documentation/ABI/removed/devfs 
            File: linux-3.14/Documentation/ABI/removed/dv1394
...

```

**Hinweis:** es ist möglich, in der bash Funktionen zu definieren. Die Syntax finden Sie z.B. hier: http://tldp.org/LDP/abs/html/functions.html

**Nützliche Befehle:** `find`,`printf`

In [71]:
chmod +x structure.sh
./structure.sh

FILE : ./Hausaufgabe02.ipynb
DIRECTORY : ./gesetze
DIRECTORY : ./I Die Grundrechte
FILE : ./Artikel 1.txt
FILE : ./Artikel 2.txt
FILE : ./Artikel 3.txt
FILE : ./Artikel 4.txt
FILE : ./Artikel 5.txt
FILE : ./Artikel 6.txt
FILE : ./Artikel 7.txt
FILE : ./Artikel 8.txt
FILE : ./Artikel 9.txt
FILE : ./Artikel 10.txt
FILE : ./Artikel 11.txt
FILE : ./Artikel 12.txt
FILE : ./Artikel 12 a.txt
FILE : ./Artikel 13.txt
FILE : ./Artikel 14.txt
FILE : ./Artikel 15.txt
FILE : ./Artikel 16.txt
FILE : ./Artikel 16 a.txt
FILE : ./Artikel 17.txt
FILE : ./Artikel 17 a.txt
FILE : ./Artikel 18.txt
FILE : ./Artikel 19.txt
DIRECTORY : ./II Der Bund und die Länder
FILE : ./Artikel 20.txt
FILE : ./Artikel 20 a.txt
FILE : ./Artikel 21.txt
FILE : ./Artikel 22.txt
FILE : ./Artikel 23.txt
FILE : ./Artikel 24.txt
FILE : ./Artikel 25.txt
FILE : ./Artikel 26.txt
FILE : ./Artikel 27.txt
FILE : ./Artikel 28.txt
FILE : ./Artikel 29.txt
FILE : ./Artikel 30.txt
FILE : ./Artikel 31.txt
FILE : ./Artikel 32.txt
FILE : ./Arti

In [56]:
cd ./I\ Die\ Grundrechte

wc: write error
bash: cd: ./IDieGrundrechte: No such file or directory


: 1

In [69]:
cd ..


In [70]:
pwd

/home/jovyan/datascience1exercise/Exercise02
